<a href="https://colab.research.google.com/github/chenchiachi/Flood_map/blob/main/Floods_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import ee


In [3]:
ee.Authenticate()

In [4]:
ee.Initialize(project='ee-chiachichen')

In [5]:
import geemap

In [6]:
m = geemap.Map(center=[29.3055, 68.9062], zoom=6)
m

Map(center=[29.3055, 68.9062], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

In [7]:
country_name = 'Pakistan'
pre_flood_start_date = '2021-08-01'
pre_flood_end_date = '2021-09-30'
flood_start_date = '2022-08-01'
flood_end_date = '2022-09-30'

In [8]:
m = geemap.Map()

country =ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017").filter(
    ee.Filter.eq("country_na", country_name)
)

style = {"color": "black", "fillColor":"00000000"}
m.add_layer(country.style(**style), {}, country_name)
m.center_object(country, 5)

m

Map(center=[29.88406293231847, 69.19999765510337], controls=(WidgetControl(options=['position', 'transparent_b…

In [9]:
m = geemap.Map()

landsat_col_2021 = (
    ee.ImageCollection("LANDSAT/LC08/C02/T1")
    .filterDate(pre_flood_start_date, pre_flood_end_date)
    .filterBounds(country)
)
landsat_2021 = ee.Algorithms.Landsat.simpleComposite(landsat_col_2021).clipToCollection(
    country
)

vis_params = {"bands":["B6", "B5", "B4"], "max":128}
m.add_layer(landsat_2021, vis_params, "Landsat 2021")

In [10]:
landsat_col_2022 = (
    ee.ImageCollection("LANDSAT/LC08/C02/T1")
    .filterDate(flood_start_date, flood_end_date)
    .filterBounds(country)
)
landsat_2022 = ee.Algorithms.Landsat.simpleComposite(landsat_col_2022).clipToCollection(
    country
)

m.add_layer(landsat_2022, vis_params, "Landsat 2022")
m.center_object(country, 5)
m

Map(center=[29.88406293231847, 69.19999765510337], controls=(WidgetControl(options=['position', 'transparent_b…

In [11]:
m = geemap.Map()
m.setCenter(68.4338, 29.3055, 6)

left_layer = geemap.ee_tile_layer(landsat_2021, vis_params, "Landsat 2021")
right_layer = geemap.ee_tile_layer(landsat_2022, vis_params, "Landsat 2022")

m.split_map(
    left_layer, right_layer, left_label="Landsat 2021", right_label = "Landsat 2022"
)
m.add_layer(country.style(**style), {}, country_name)
m


Map(center=[29.3055, 68.4338], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [12]:
ndwi_2021 = landsat_2021.normalizedDifference(["B3", "B5"]).rename("NDWI")
ndwi_2022 = landsat_2022.normalizedDifference(["B3", "B5"]).rename("NDWI")

In [13]:
m = geemap.Map()
m.setCenter(68.4338, 26.4213, 7)

ndwi_vis = {"min": -1, "max": 1, "palette": "ndwi"}

left_layer = geemap.ee_tile_layer(ndwi_2021, ndwi_vis, "Landsat 2021")
right_layer = geemap.ee_tile_layer(ndwi_2022, ndwi_vis, "Landsat 2022")

m.split_map(
    left_layer, right_layer, left_label="NDWI 2021", right_label = "NDWI 2022"
)
m.add_layer(country.style(**style), {}, country_name)
m

Map(center=[26.4213, 68.4338], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [14]:
threshold = 0.1
water_2021 = ndwi_2021.gt(threshold)
water_2022 = ndwi_2022.gt(threshold)

In [15]:
m = geemap.Map()
m.setCenter(68.4338, 26.4213, 7)

m.add_layer(landsat_2021, vis_params, "Landsat 2021", False)
m.add_layer(landsat_2022, vis_params, "Landsat 2022", False)

left_layer = geemap.ee_tile_layer(
    water_2021.selfMask(), {"palette": "blue"}, "Water 2021"
)

right_layer = geemap.ee_tile_layer(
    water_2022.selfMask(), {"palette": "red"}, "Water 2022"
)

m.split_map(left_layer, right_layer, left_label="Water 2021", right_label="Water 2022")
m.add_layer(country.style(**style), {}, country_name)
m

Map(center=[26.4213, 68.4338], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [16]:
flood_extent = water_2022.subtract(water_2021).gt(0).selfMask()

In [44]:
m = geemap.Map()
m.setCenter(68.4338, 26.4213, 7)

m.add_layer(landsat_2021, vis_params, "Landsat 2021", True)
m.add_layer(landsat_2022, vis_params, "Landsat 2022", True)

left_layer = geemap.ee_tile_layer(
    water_2021.selfMask(), {"palette": "blue"}, "Water 2021"
)

right_layer = geemap.ee_tile_layer(
    water_2022.selfMask(), {"palette": "cyan"}, "Water 2022"
)

m.split_map(left_layer, right_layer, left_label="Water 2021", right_label="Water 2022")
m.add_layer(country.style(**style), {}, country_name)
m

Map(center=[26.4213, 68.4338], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [18]:
area_2021 = geemap.zonal_stats(
    water_2021.selfMask(),country, scale=1000, stat_type ="SUM", return_fc = True
)
geemap.ee_to_df(area_2021)

Computing statistics ...


,abbreviati,country_co,country_na,sum,wld_rgn
0,Pak.,PK,Pakistan,4205.678431,S Asia


In [19]:
area_2022 = geemap.zonal_stats(
    water_2022.selfMask(),country, scale=1000, stat_type ="SUM", return_fc = True
)
geemap.ee_to_df(area_2022)

Computing statistics ...


,abbreviati,country_co,country_na,sum,wld_rgn
0,Pak.,PK,Pakistan,13145.027451,S Asia


In [20]:
flood_area = geemap.zonal_stats(
    flood_extent.selfMask(),country, scale=1000, stat_type ="SUM", return_fc = True
)
geemap.ee_to_df(flood_area)

Computing statistics ...


,abbreviati,country_co,country_na,sum,wld_rgn
0,Pak.,PK,Pakistan,11065.72549,S Asia


In [24]:
from google.colab import files
files.download("flood_map.html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>